In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.5 MB/s 
     |████████████████████████████████| 6.5 MB 30.1 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 53.9 MB/s 
     |████████████████████████████████| 895 kB 49.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import torch
import gc
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def score_to_class(score):
  if score==1:
    return 0
  elif score==1.5 :
    return 1
  elif score== 2 :
    return 2
  elif score==2.5 :
    return 3
  elif score==3 :
    return 4
  elif score==3.5 :
    return 5
  elif score==4 :
    return 6

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path = F"/content/gdrive/My Drive/Colab Notebooks/4. Argument Strength Model Training/AS_Label_data.xlsx" 
path_2=F"/content/gdrive/My Drive/Colab Notebooks/3. Organisation Score Model Training/train_test_5_fold.xlsx"

Mounted at /content/gdrive


In [ ]:
df=pd.read_excel(path,sheet_name='n_gram')
df=df.fillna(0)

For performance reasons, we'll only use 2,000 sentences from the dataset

In [ ]:
# 5 fold train test split using excel file 
fold_1=pd.read_excel(path_2,sheet_name='Fold1')
fold_1.fillna('')
fold_1_train=pd.merge(fold_1[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_1_test=(pd.merge(fold_1[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_1_train),len(fold_1_test))

fold_2=pd.read_excel(path_2,sheet_name='Fold2')
fold_2.fillna('')
fold_2_train=pd.merge(fold_2[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_2_test=(pd.merge(fold_2[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_2_train),len(fold_2_test))

fold_3=pd.read_excel(path_2,sheet_name='Fold3')
fold_3.fillna('')
fold_3_train=pd.merge(fold_3[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_3_test=(pd.merge(fold_3[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_3_train),len(fold_3_test))

fold_4=pd.read_excel(path_2,sheet_name='Fold4')
fold_4.fillna('')
fold_4_train=pd.merge(fold_4[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_4_test=(pd.merge(fold_4[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_4_train),len(fold_4_test))

fold_5=pd.read_excel(path_2,sheet_name='Fold5')
fold_5.fillna('')
fold_5_train=pd.merge(fold_5[['train_file']],df,left_on='train_file',right_on='Essay',how='left')
fold_5_test=(pd.merge(fold_5[['test_file']],df,left_on='test_file',right_on='Essay',how='left')).dropna()
print(len(fold_5_train),len(fold_5_test))

799 201
800 200
799 201
801 199
801 199


We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model. 

In [ ]:
# For DistilBERT:
# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# model_class, tokenizer_class, pretrained_weights = (ppb.BertForSequenceClassification, ppb.BertTokenizer, 'bert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
def get_hidden_layer(batch_1):
  tokenized = batch_1['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=512,truncation=True)))  # -- changed from 300 to 512 for testing on upb server
  max_len = 0
  for i in tokenized.values:
      if len(i) > max_len:
          max_len = len(i)
  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
  attention_mask = np.where(padded != 0, 1, 0)
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)
  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)
  features = last_hidden_states[0][:,0,:].numpy()
  X_full = np.concatenate((batch_1.iloc[:,4:].to_numpy(), features), axis = 1)
  print(len(batch_1.iloc[:,4:].to_numpy()))
  print(features.shape)
  print(X_full.shape)
  return X_full

The labels indicating which sentence is positive and negative now go into the `labels` variable

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

In [ ]:
gc.collect()

12536

In [ ]:
train_features=get_hidden_layer(fold_1_train)
test_features=get_hidden_layer(fold_1_test)
train_labels=fold_1_train.ArgumentStrength_Score
test_labels=fold_1_test.ArgumentStrength_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_1_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score : ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
gc.collect()

799
(799, 768)
(799, 2667)
201
(201, 768)
(201, 2667)
RF score :  0.02104667572600538
MAE :  0.39900497512437805
MSE :  0.241592039800995


72

In [ ]:
train_features=get_hidden_layer(fold_2_train)
test_features=get_hidden_layer(fold_2_test)
train_labels=fold_2_train.ArgumentStrength_Score
test_labels=fold_2_test.ArgumentStrength_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_2_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score : ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
gc.collect()

800
(800, 768)
(800, 2667)
200
(200, 768)
(200, 2667)
RF score :  0.09817691508906456
MAE :  0.363
MSE :  0.19579999999999997


72

In [ ]:
import joblib
joblib.dump(rf, "random_forest_arg.joblib")

['random_forest_arg.joblib']

In [ ]:
train_features=get_hidden_layer(fold_3_train)
test_features=get_hidden_layer(fold_3_test)
train_labels=fold_3_train.ArgumentStrength_Score
test_labels=fold_3_test.ArgumentStrength_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_3_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score : ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
gc.collect()

799
(799, 768)
(799, 2667)
201
(201, 768)
(201, 2667)
RF score :  0.03695610905525504
MAE :  0.4059701492537315
MSE :  0.24955223880597013


72

In [ ]:
train_features=get_hidden_layer(fold_4_train)
test_features=get_hidden_layer(fold_4_test)
train_labels=fold_4_train.ArgumentStrength_Score
test_labels=fold_4_test.ArgumentStrength_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_4_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score : ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
gc.collect()

801
(801, 768)
(801, 2667)
199
(199, 768)
(199, 2667)
RF score :  0.06941148528931795
MAE :  0.3824120603015075
MSE :  0.22206030150753767


72

In [ ]:
train_features=get_hidden_layer(fold_5_train)
test_features=get_hidden_layer(fold_5_test)
train_labels=fold_5_train.ArgumentStrength_Score
test_labels=fold_5_test.ArgumentStrength_Score
rf=RandomForestRegressor(n_estimators=150)
rf.fit(train_features, train_labels)
test_label_numpyarray=test_labels.to_numpy()
y_pred = rf.predict(test_features).round(decimals=1)
fold_5_rf_score=mean_absolute_error(test_labels, y_pred)
print('RF score : ',rf.score(test_features, test_labels))
print('MAE : ',mean_absolute_error(test_labels, y_pred))
print('MSE : ',mean_squared_error(test_labels, y_pred))
gc.collect()

801
(801, 768)
(801, 2667)
199
(199, 768)
(199, 2667)
RF score :  0.03990126657677717
MAE :  0.4010050251256282
MSE :  0.24653266331658294


72

In [ ]:
print((fold_1_rf_score+fold_2_rf_score+fold_3_rf_score+fold_4_rf_score+fold_5_rf_score)/5)

0.3902784419610491


In [ ]:
# def get_score_buckets(y_pred):
#   if y_pred<1.5:
#     return 1
#   elif y_pred>=1.5 and y_pred<=1.75:
#     return 1.5
#   elif y_pred>1.75 and y_pred<=2.25:
#     return 2
#   elif y_pred>2.25 and y_pred<=2.75:
#     return 2.5
#   elif y_pred>2.75 and y_pred<=3.25:
#     return 3
#   elif y_pred>3.25 and y_pred<3.5:
#     return 3.5
#   elif y_pred>=3.5:
#     return 4

In [ ]:
# y_pred_bucket=pd.DataFrame(y_pred)
# y_pred_bucket.columns=['y_pred']
# y_pred_bucket['bucket']=y_pred_bucket.y_pred.apply(get_score_buckets)
# y_pred_numpy_bucket=y_pred_bucket[['bucket']].to_numpy()
# y_pred_bucket

In [ ]:
# for i in range(0,len(test_label_numpyarray)):
#   if (test_label_numpyarray[i]==y_pred_numpy_bucket[i]) :
#     count = count + 1
#   total = total + 1
# accuracy = count/total*100
# print('RF regressor accuracy : ',accuracy)


In [ ]:
# y_pred_class=y_pred_bucket.bucket.apply(score_to_class)
# y_test_label=test_labels.apply(score_to_class)
# # y_test_label

In [ ]:
# from sklearn.metrics import confusion_matrix
# conf_mat = confusion_matrix(y_test_label, y_pred_class,labels=[0,1,2,3,4,5,6])
# print(conf_mat)

In [ ]:
# import joblib
# joblib.dump(rf, "Arg_Strength_random_forest_75_25_feature.joblib")

# **svm performance analysis**

In [ ]:
#Selecting the value of C for SVM

# from sklearn.svm import SVR
# from sklearn.metrics import mean_squared_error,r2_score

# c = [ 0.1, 1, 10, 100, 1000, 10000, 100000,1000000,10000000,100000000]
# c_val = ['0.1', '1', '10','100','10*2','10*3','10*4','10*5','10*6','10*7']
# MSE = []
# i = 0
# while i<10:
#   svr = SVR(C=c[i])
#   svr.fit(train_features, train_labels)
#   y_pred = svr.predict(test_features)
#   MSE_svc = mean_squared_error(y_pred, test_labels)
#   MSE.append(MSE_svc)
#   i = i+1
# plt.plot(c_val,MSE)
# plt.xlabel('C')
# plt.ylabel('MSE')
# plt.show()

In [ ]:
# # exact score match for SVR

# from sklearn.svm import SVR
# svr = SVR(C=1000000)
# svr.fit(test_features, test_labels)
# svr_y_pred=svr.predict(test_features).round()


# print('svr score : ',svr.score(test_features, test_labels))
# print('MAE : ',mean_absolute_error(test_labels, svr_y_pred))
# print('MSE : ',mean_squared_error(test_labels, svr_y_pred))

In [ ]:
# svr_y_pred_bucket=pd.DataFrame(svr_y_pred)
# svr_y_pred_bucket.columns=['y_pred']
# svr_y_pred_bucket['bucket']=svr_y_pred_bucket.y_pred.apply(get_score_buckets)
# svr_y_pred_numpy_bucket=svr_y_pred_bucket[['bucket']].to_numpy()
# # svr_y_pred_bucket

In [ ]:
# for i in range(0,len(test_label_numpyarray)):
#   if (test_label_numpyarray[i]==svr_y_pred_numpy_bucket[i]) :
#     count = count + 1
#   total = total + 1
# accuracy = count/total*100
# print('SVM regressor accuracy : ',accuracy)

# conf_mat = confusion_matrix(y_test_label, svr_y_pred_numpy_bucket,labels=[0,1,2,3,4,5,6])
# print(conf_mat)

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:
# from sklearn.dummy import DummyClassifier
# clf = DummyClassifier()

# scores = cross_val_score(clf, train_features, train_labels)
# print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# df

So our model clearly does better than a dummy classifier. But how does it compare against the best models?

## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.



And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue). You can also go back and switch from distilBERT to BERT and see how that works.